In [9]:
%pylab inline
import sys
sys.path.insert(0, "..")
from importlib import reload
import numpy as np
import opt
from opt import ls
import time
import scipy.linalg
from sklearn import metrics
import utils
from utils import exputil
import distributed
from distributed import sharded_matrix
from distributed.distributed import fast_kernel_column_blocks_get
reload(distributed.distributed)
import time

Populating the interactive namespace from numpy and matplotlib


In [60]:
! aws s3 ls s3://vaishaalpywrenlinalg/pywren.linalg/

                           PRE XTX(mnist_test_1)/
                           PRE XTX(mnist_test_1)_column_block_0/
                           PRE XTX(mnist_test_1)_column_block_1/
                           PRE XTX(mnist_test_1)_column_block_10/
                           PRE XTX(mnist_test_1)_column_block_11/
                           PRE XTX(mnist_test_1)_column_block_12/
                           PRE XTX(mnist_test_1)_column_block_13/
                           PRE XTX(mnist_test_1)_column_block_14/
                           PRE XTX(mnist_test_1)_column_block_2/
                           PRE XTX(mnist_test_1)_column_block_3/
                           PRE XTX(mnist_test_1)_column_block_4/
                           PRE XTX(mnist_test_1)_column_block_5/
                           PRE XTX(mnist_test_1)_column_block_6/
                           PRE XTX(mnist_test_1)_column_block_7/
                           PRE XTX(mnist_test_1)_column_block_8/
                           PRE XTX(

In [119]:
%time X_train_sharded = sharded_matrix.ShardedMatrix("X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017", bucket="vaishaalpywrenlinalg")

CPU times: user 60 ms, sys: 12 ms, total: 72 ms
Wall time: 114 ms


In [59]:
X_train_sharded.shape

(1281167, 18432)

In [120]:
%time X_test_sharded = sharded_matrix.ShardedMatrix("X_test_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017", bucket="vaishaalpywrenlinalg")

CPU times: user 68 ms, sys: 12 ms, total: 80 ms
Wall time: 160 ms


In [121]:
y_test = np.load("/home/ubuntu/imagenet-data/scrambled_test_labels.npy")
y_train = np.load("/home/ubuntu/imagenet-data/scrambled_train_labels.npy")

In [37]:
def compute_median(X, num_samples):
    idxs_0 = np.random.choice(X.shape[0], num_samples, replace=False)
    idxs_1 = np.random.choice(X.shape[0], num_samples, replace=False)
    dist = []
    for i in range(num_samples):
        x = X[idxs_0[i]]
        y = X[idxs_1[i]]
        dist.append(np.linalg.norm(x)/np.sqrt(x.shape[0]))
    return np.power(np.median(dist),2), np.power(np.max(dist), 2)
    

In [122]:
%time X_train = fast_kernel_column_blocks_get(X_train_sharded, col_blocks=[0], mmap_loc="/dev/shm/X_train", dtype="float64")

CPU times: user 0 ns, sys: 25.4 s, total: 25.4 s
Wall time: 7min 23s


2048 0 12 (848, 2048, 3, 3) (848, 2048)
X_FULL PART SHAPE  (848, 2048)
X FULL SHAPE  (50000, 2048)
could not broadcast input array from shape (848,2048,3,3) into shape (848,2048)
2048 0 2 (4096, 2048, 3, 3) (4096, 2048)
X_FULL PART SHAPE  (4096, 2048)
X FULL SHAPE  (50000, 2048)
could not broadcast input array from shape (4096,2048,3,3) into shape (4096,2048)
2048 0 11 (4096, 2048, 3, 3) (4096, 2048)
X_FULL PART SHAPE  (4096, 2048)
X FULL SHAPE  (50000, 2048)
could not broadcast input array from shape (4096,2048,3,3) into shape (4096,2048)
2048 0 8 (4096, 2048, 3, 3) (4096, 2048)
X_FULL PART SHAPE  (4096, 2048)
X FULL SHAPE  (50000, 2048)
could not broadcast input array from shape (4096,2048,3,3) into shape (4096,2048)
2048 0 6 (4096, 2048, 3, 3) (4096, 2048)
X_FULL PART SHAPE  (4096, 2048)
X FULL SHAPE  (50000, 2048)
could not broadcast input array from shape (4096,2048,3,3) into shape (4096,2048)
2048 0 0 (4096, 2048, 3, 3) (4096, 2048)
X_FULL PART SHAPE  (4096, 2048)
X FULL SHAPE  (

ValueError: could not broadcast input array from shape (4096,2048,3,3) into shape (4096,2048)

In [18]:
%time X_train_sharded = sharded_matrix.ShardedMatrix("X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_4_29_2017",\
                                                     shard_size_0=4096,\
                                                     shard_size_1=9216,\
                                                     bucket="vaishaalpywrenlinalg",\
                                                     reshard=True,\
                                                     n_jobs=32,\
                                                     data=X_train)


Sharding matrix..... of shape (50000, 18432)
00 409649152 4096  12288491524096 4096819250000  2048045056163844096081924096081922457616384 8192   1638450000         3276824576204803686432768368644505628672122880 921609216  245764915220480450561228845056122882867220480         28672 9216  00         368642867224576409603686440960491523276816384 9216 18432 18432  092160921600921609216         32768
18432
9216
92169216         921692169216009216092169216 0
 
 

9216184329216184329216921618432921618432        921618432184321843292169216








1843218432184329216









CPU times: user 1min 58s, sys: 1min 22s, total: 3min 21s
Wall time: 1min 24s


In [38]:
compute_median(X_train, 1000)

(0.10451829955360722, 0.72024884937146538)

In [30]:
np

CPU times: user 28 ms, sys: 1.63 s, total: 1.66 s
Wall time: 8.98 s


In [123]:
X_train.dtype

dtype('float64')

In [124]:
%time XTX = X_train.T.dot(X_train)

CPU times: user 7h 55min 58s, sys: 8h 56min 5s, total: 16h 52min 4s
Wall time: 17min 18s


In [40]:
y_train_enc = np.eye(1000)[y_train.astype('int')]

In [128]:
%time XTy = X_train.T.dot(y_train_enc)

CPU times: user 49min 46s, sys: 1h 29min 19s, total: 2h 19min 5s
Wall time: 2min 21s


In [126]:
XTX += np.eye(XTX.shape[0])

In [129]:
%time model = scipy.linalg.solve(XTX, XTy)

CPU times: user 9min 19s, sys: 1min 34s, total: 10min 54s
Wall time: 24.9 s


In [ ]:
y_train_pred = X_train.dot(model)

In [ ]:
y_test_pred = X_test.dot(model)

In [52]:
print("Train Accuracy ", metrics.accuracy_score(y_train, np.argmax(y_train_pred, axis=1)))
print("Test Accuracy ", metrics.accuracy_score(y_test, np.argmax(y_test_pred, axis=1)))

Train Accuracy  0.401566696613
Test Accuracy  0.1888


In [97]:
K = sharded_matrix.ShardedSymmetricMatrix("rbf(XXT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017), 4.0)", bucket="vaishaalpywrenlinalg")

In [90]:
K.get

array([[ 0.9999997 ,  0.1047346 ,  0.06749739, ...,  0.10831299,
         0.02672276,  0.15164699],
       [ 0.1047346 ,  1.00000131,  0.05209133, ...,  0.06496691,
         0.01898118,  0.10122403],
       [ 0.06749739,  0.05209133,  1.        , ...,  0.05777011,
         0.0195615 ,  0.07338814],
       ..., 
       [ 0.10831299,  0.06496691,  0.05777011, ...,  0.99999946,
         0.01671762,  0.12393676],
       [ 0.02672276,  0.01898118,  0.0195615 , ...,  0.01671762,
         1.        ,  0.02681992],
       [ 0.15164699,  0.10122403,  0.07338814, ...,  0.12393676,
         0.02681992,  0.99999988]], dtype=float32)

In [115]:
K.get_block(0,0)

NameError: name 'K' is not defined

In [114]:
list_of_parts = open("../files").readlines()

In [71]:
add_sentinels = list(filter(lambda x: "add" in x, map(lambda x: x.strip(), list_of_parts)))


In [96]:
add_sentinel_split = list(map(get_parts_from_add_sentinel, add_sentinels))

In [109]:
add_sentinel_split.index((0,28))

201

In [113]:
list_of_parts[49141:][202]

'2017-04-29 23:03:59        201 add_sentinel(0, 28, 99c2a2f17f0af87214d7da2748d793dcd8704dd1, d9b7b91e94fd6d51627b5c52e1689063312abfb6)\n'

In [103]:
import collections
print([item for item, count in collections.Counter(add_sentinel_split).items() if count > 1])

[(0, 28), (0, 29), (0, 30), (0, 31), (103, 157), (103, 158), (103, 159), (103, 160), (126, 155), (126, 156), (126, 157), (126, 158), (132, 154), (132, 155), (132, 156), (132, 157), (132, 258), (132, 259), (132, 260), (132, 261), (138, 261), (138, 262), (138, 263), (138, 264), (150, 175), (150, 176), (150, 177), (150, 178), (160, 236), (160, 237), (160, 238), (160, 239), (162, 189), (162, 190), (162, 191), (162, 192), (216, 256), (216, 257), (216, 258), (216, 259), (237, 266), (237, 267), (237, 268), (237, 269), (28, 100), (28, 101), (28, 98), (28, 99), (30, 163), (30, 164), (30, 165), (30, 166), (31, 261), (31, 262), (31, 263), (31, 264), (38, 91), (38, 92), (38, 93), (38, 94), (51, 247), (51, 248), (51, 249), (51, 250), (53, 178), (53, 179), (53, 180), (53, 181), (54, 95), (54, 96), (54, 97), (54, 98), (60, 110), (60, 111), (60, 112), (60, 113), (63, 277), (63, 278), (63, 279), (63, 280), (64, 272), (64, 273), (64, 274), (64, 275), (66, 185), (66, 186), (66, 187), (66, 188), (85, 134)

In [90]:
def get_parts_from_add_sentinel(sentinel):
    first = int(sentinel.split("add_sentinel")[1].split("(")[1].split(",")[0].strip())
    second = int(sentinel.split("add_sentinel")[1].split("(")[1].split(",")[1].strip())
    return first, second


In [118]:
get_parts_from_add_sentinel(add_sentinels[3])

(0, 100)

In [89]:
second

0